In [1]:
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from random import randrange, uniform
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

%matplotlib qt

#FrankeFunction - for simulation of data
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x - 2)**2)- 0.25*((9*y - 2)**2))
    term2 = 0.75*np.exp(-((9*x + 1)**2)/49.0 - 0.1*(9*y + 1))
    term3 = 0.5*np.exp(-(9*x - 7)**2/4.0 - 0.25*((9*y - 3)**2))
    term4 = -0.2*np.exp(-(9*x - 4)**2 - (9*y - 7)**2)
    return term1 + term2 + term3 + term4 + 0.1*np.random.randn(x.shape[0], x.shape[1])

#Ordinary Least Squared function
def ols(x, y, z):
    #x: vector of size(n, 1)
    #y: vector of size(n,1)
    # z: vector of size(n,1)

    xyb = np.c_[np.ones_like(x), x, y, x*x, y*y, x*y]
    beta = np.linalg.inv(xyb.T.dot(xyb)).dot(xyb.T).dot(z)
    return beta

#Mean Squared Error
def MSE(zReal, zPredicted):
    mse = np.mean((zReal-zPredicted)**2)
    return mse           


#Mean value of the function
def Mean(z):
    meanValue = np.mean(z)
    return meanValue


#R2 score function
def R2(zReal, zPredicted):
    meanValue = Mean(zReal)
    numerator = np.sum((zReal - zPredicted)**2)
    denominator = np.sum((zReal - meanValue)**2)
    result = 1 - (numerator/denominator)
    return result

#Purpose of k-fold validation is to divide all the samples in k groups of samples equal sizes. 
#The prediction function is learned using k - 1 folds. We leave the last fold/subset for test.
def kfold(x,y, z, splits):
    listTotal = []
    listx = []
    listy = []
    listz = []
    inter = len(x)//splits
    remainder = len(x)%splits
    
    for i in range(splits):
        if(i != splits-1):
            listx.append(x[inter*i:inter*(i+1)])
            listy.append(y[inter*i:inter*(i+1)])
            listz.append(z[inter*i:inter*(i+1)])
        else:
            listx.append(x[inter*(splits-1):inter*splits + remainder])
            listy.append(y[inter*(splits-1):inter*splits + remainder])
            listz.append(z[inter*(splits-1):inter*splits + remainder])
    listTotal.append(listx)
    listTotal.append(listy)
    listTotal.append(listz)
    return listTotal
    #return (listx, listy, listz)


In [ ]:
############################################
#Training and testing with k-fold validation
############################################

# Data generated for training and test purposes
X = np.random.rand(10,1)
Y = np.random.rand(10,1)
z = FrankeFunction(X, Y)

#Splitting data into training subsets and one test subset
allData = kfold(X,Y, z, 5) # last parameter refers to amount of folds/subsets
#np.asarray(mylist)
xSubsets = allData[0]
ySubsets = allData[1]
zSubsets = allData[2] 

#Getting beta from ols-function for 5 models from my subsets
beta1 = ols(xSubsets[0], ySubsets[0], zSubsets[0])
beta2 = ols(xSubsets[1], ySubsets[1], zSubsets[1])
beta3 = ols(xSubsets[2], ySubsets[2], zSubsets[2])
beta4 = ols(xSubsets[3], ySubsets[3], zSubsets[3])
#Test subset
#xSubsets[4], ySubsets[4], zSubsets[4])



In [2]:
#For plotting
%matplotlib qt
fig = plt.figure(figsize = (10,10))
ax = fig.gca(projection='3d')

#Defining training data
X = np.random.rand(100,1)
Y = np.random.rand(100, 1)
z = FrankeFunction(X, Y)

#Making a model from Frankes function
z = FrankeFunction(X, Y)

#Getting beta from ols-function
beta = ols(X, Y, z)

#Chose test data.
X_test = np.random.rand(100, 1)
Y_test = np.random.rand(100, 1)
X_test, Y_test = np.meshgrid(X_test,Y_test) # makes X and Y (100,100)-matrix and maps each x agains each y
X_flat_t = X_test.reshape(-1,1)
Y_flat_t = Y_test.reshape(-1,1)

xybnew = np.c_[np.ones_like(X_flat_t), X_flat_t, Y_flat_t, X_flat_t*Y_flat_t, Y_flat_t*Y_flat_t, X_flat_t*Y_flat_t] 
zpredict = xybnew.dot(beta).reshape(100, 100)
zreal = FrankeFunction(X_test, Y_test)
#zreal = FrankeFunction(X_test, Y_test).reshape(100, 100)

#Plot surface - predicted
surfpredict = ax.plot_surface(X_test, Y_test,zpredict,cmap=cm.coolwarm, linewidth=0, antialiased=False)
#surfreal = ax.plot_surface(X_test, Y_test, zreal,cmap=cm.coolwarm, linewidth=0, antialiased=False)
points = ax.scatter( X_test, Y_test, zreal) #Plot of the real values
    
#Customise the z axis
ax.set_zlim(-0.10, 1.40)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    

#Add a color bar which maps values to colors
fig.colorbar(surfpredict, shrink=0.5, aspect=5)
plt.show()


print ("Mean squared error: %.2f" % MSE(zreal, zpredict))
print("R2 score: %.2f" % R2(zreal, zpredict))

Mean squared error: 0.03
R2 score: 0.72
